## Bias Detection Model Fine-tuning Example

In [ ]:
# import statements
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
# from tqdm import tqdm
from tqdm.auto import tqdm
tqdm.pandas()

from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
import torch
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence
from torch.optim.lr_scheduler import StepLR
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

### Data Loading and Preperation

In [ ]:
# Example: Load one of the .xlsx files
df = pd.read_csv('./data/MBIC/final_labels_MBIC.csv', sep=';', encoding='utf-8')
df.drop(columns=['type', 'topic', 'outlet', 'news_link'], inplace=True)

df.dropna(inplace=True)  # Remove missing values
df.reset_index(inplace=True)

# clean the 
# remove no agreement rows
df = df[df['label_bias'] != 'No agreement']

# Convert categorical labels to numerical labels
le = LabelEncoder()
le.fit(['Non-biased', 'Biased'])
df['label_bias_numeric'] = le.transform(df['label_bias'])

**NOTE: the label encoder marks Biased as 0, Non-biased as 1**

In [1]:
# load trained model
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
import torch
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence
from torch.optim.lr_scheduler import StepLR

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("distilroberta-MBIC-2-tokenizer")
model = AutoModelForSequenceClassification.from_pretrained("distilroberta-MBIC-2")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the classification function
def classify_text(text):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)

    ## Move the tokenized inputs to the same device as the model
    inputs = {k: v.to(device) for k, v in inputs.items()}

    model.to(device)
    
    # Forward pass through the model
    outputs = model(**inputs)
    
    # Get the predicted label
    predicted_label = outputs.logits.argmax().item()
    
    return predicted_label


c:\Users\13343\anaconda3\envs\Dbias\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:

def collate_fn(batch):
    texts = [item['input_ids'].squeeze(0) for item in batch]  # Remove the extra dimension
    labels = torch.tensor([item['labels'] for item in batch])
    texts_padded = pad_sequence(texts, batch_first=True, padding_value=0)  # Set batch_first to True
    
    return {'input_ids': texts_padded, 'labels': labels}


# 1. Split the dataframe into training and validation sets
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

# 2. Preprocess the text data and convert it into numerical features
class TextDataset(Dataset):
    def __init__(self, df):
        self.texts = df['text'].tolist()
        self.labels = df['label_bias_numeric'].tolist()
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        inputs = self.tokenizer(text, return_tensors="pt", padding=True, truncation=True)
        inputs = {key: tensor.squeeze(0) for key, tensor in inputs.items()}  # Remove the extra batch dimension
        inputs['labels'] = torch.tensor(label)

        return inputs


train_dataset = TextDataset(train_df)
val_dataset = TextDataset(val_df)

# 3. Define the training loop and optimization process
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
scheduler = StepLR(optimizer, step_size=3, gamma=0.1)

# Calculate class weights for imbalanced distribution
class_counts = train_df['label_bias_numeric'].value_counts().to_list()
class_weights = [sum(class_counts) / c for c in class_counts]
weights = torch.tensor(class_weights, dtype=torch.float).to(device)

# Use the weights in the loss function
# This can help the model pay more attention to underrepresented classes, potentially improving recall.
loss_fn = torch.nn.CrossEntropyLoss(weight=weights)

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True, collate_fn=collate_fn)
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False, collate_fn=collate_fn)

# 4. Train the model on the training set
num_epochs = 3

for epoch in tqdm(range(num_epochs)):
    model.train()
    train_loss = 0.0

    for batch in train_dataloader:
        inputs = {k: v.to(device) for k, v in batch.items()}
        optimizer.zero_grad()
        outputs = model(**inputs)
        loss = loss_fn(outputs.logits, inputs['labels'])
        loss.backward()
        optimizer.step()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0) # clipping

        train_loss += loss.item()
        
    scheduler.step()

    # 5. Evaluate the model on the validation set
    model.eval()
    val_loss = 0.0
    val_correct = 0
    

    with torch.no_grad():
        for batch in val_dataloader:
            inputs = {k: v.to(device) for k, v in batch.items()}

            outputs = model(**inputs)
            loss = loss_fn(outputs.logits, inputs['labels'])
            val_loss += loss.item()

            predicted_labels = outputs.logits.argmax(dim=1)
            val_correct += (predicted_labels == inputs['labels']).sum().item()

    train_loss /= len(train_dataloader)
    val_loss /= len(val_dataloader)
    val_accuracy = val_correct / len(val_dataset)

    print(f"Epoch {epoch+1}/{num_epochs}:")
    print(f"  Train Loss: {train_loss:.4f}")
    print(f"  Val Loss: {val_loss:.4f}")
    print(f"  Val Accuracy: {val_accuracy:.4f}")

In [ ]:
# Performance evaluation on unseen data (to avoid the similar language style from same data source)
sample_data = pd.read_csv('./data/generated_data.csv', sep=',', encoding='utf-8')

# le = LabelEncoder()
# le.fit(['Non-biased', 'Biased'])
sample_data['label_bias_numeric'] = le.transform(sample_data['Label'])

# Create a new column 'prediction' in df
sample_data['prediction'] = 0

# Apply the classify_text_apply function on each row of 'Text' column with tqdm progress bar
sample_data['prediction'] = sample_data['Text'].progress_apply(classify_text)

accuracy = accuracy_score(sample_data['label_bias_numeric'], sample_data['prediction'])
f1 = f1_score(sample_data['label_bias_numeric'], sample_data['prediction'])
precision = precision_score(sample_data['label_bias_numeric'], sample_data['prediction'])
recall = recall_score(sample_data['label_bias_numeric'], sample_data['prediction'])

print("Accuracy:", accuracy)
print("F1 Score:", f1)
print("Precision:", precision)
print("Recall:", recall)